In [321]:
# Import needed libaries
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# read in dataset and remove previouly set multiindex
df = pd.read_csv("process_data_df10.csv", header=[0,1], na_values=['nan'],
sep=',',low_memory=False)

# delet column without any information 
df.drop(['Unnamed: 0_level_0'],axis=1, inplace = True)
df = df.droplevel(level=0,axis=1)

# drop na and duplicates from qc_serien number since following assumption is made:
# every qc has one serial number, if it occurs twice the sample has been tested twice, since there are
# less than 1000 duplicates (less than 0.5 % of whole dataset) to keep them
df = df.dropna(subset = ['qc_serien_nummer']).drop_duplicates(subset = ['qc_serien_nummer'])#.set_index('qc_serien_nummer')


df = df[df['qc_salzrckhalt']!=0]
#df.to_pickle('test.pkl')

/home/elron/anaconda3/envs/da/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


# Updating Reference and Richtwerte naming
- Renaming columns named richtwert into ref for easier filtering

In [322]:
# renaming columns named richtwert into ref for better filtering


df.rename(columns={'pa_cm-pda_gewichts-%_richtwert_ref_low':'pa_cm-pda_gewichts-%_ref_low'}, inplace=True)
df.rename(columns={'pa_cm-pda_gewichts-%_richtwert_ref_high':'pa_cm-pda_gewichts-%_ref_high'}, inplace=True)
df.rename(columns={'pa_chlorkonzentration_richtwert_low_ppm':'pa_chlorkonzentration_ref_low_ppm'}, inplace=True)
df.rename(columns={'pa_chlorkonzentration_richtwert_high_ppm':'pa_chlorkonzentration_ref_high_ppm'}, inplace=True)

In [323]:
df1 = df.copy()

In [324]:
result = pd.concat([df1.filter(regex = '_ref'),df1.filter(regex = '_richtwert_')], axis = 1, sort = False)
result1 = result.loc[:,~result.columns.duplicated()]
result2 = result1.join(df['p_product'])
result1.drop('pa_ref', axis=1, inplace = True)

In [325]:
Richtwerte = list(result1.columns)#.remove('p_product')


In [326]:
df1 = df1.drop(result1, axis = 1) #drop ref temporaly

# Make mean columns

In [327]:
# Multiple Measurment columns
meanList = ['pa_cm-pda','pa_chlorkonzentration','pa_air-knife_vor_amin_bad','airknife_vor_amin_trockner',
            'pa_amin-trockner_feuchtigkeit_danfugt_bs',
            'pa_amin-trockner_feuchtigkeit_danfugt_as',
            'pa_amin-trockner_feuchtigkeit_cofely',
            'ps_dicke','ps_bad_temperatur',
            'ps_auftragswerk_feuchtigkeit',
            'sc_d_links_werte',
            'sc_d_rechts_werte']

# Funktion to drop multiple measurment values
def MeanColumnCreate(meanList, df):
  for i in meanList:
      filterdf = df.filter(regex = i)
      name = i+"_mean"
      df[name] = filterdf.iloc[:].mean(axis = 1)
      df.drop(filterdf, axis = 1, inplace = True)
    
MeanColumnCreate(meanList, df1)

# Rename air-knife vor amin bad mean to airknife ....
df1.rename(columns={'pa_air-knife_vor_amin_bad_mean':'pa_airknife_vor_amin_bad_mean'}, inplace=True)

In [339]:
df2 = df1.copy()

In [340]:
# rejoin refvalues back to dataframe
df2 = df2.join(result1)

# Lists for name pattern search
- List has been created manually

In [341]:
# List of features that have low and high reference values
RefDouble =  ['pa_cm-pda','pa_chlorkonzentration',
            'airknife_vor_amin_trockner',
            'pa_amin-trockner_feuchtigkeit_danfugt_bs',
            'pa_amin-trockner_feuchtigkeit_danfugt_as',
            'pa_amin-trockner_feuchtigkeit_cofely',
            'pa_vertikale_feuchte_oben',
            'pa_temperatur_n-decan-lsg', 
            'pa_temperatur_alkali-lsg']

# List of features that have only one reference value
RefSingle = ['pa_bahngeschwindigkeit', 'ct1',
                 'pa_staubsauger',
                 'pa_airknife_vor_amin_bad',
                 'pa_amin-trockner_temperatur_danfugt',
                 'pa_decan-trockner_geblase']
                 

In [342]:
df3 = df2.copy()

# Function for calculating with low and high ref value

In [343]:
def doubleRef(df, RefListDouble):
    for i in RefListDouble:
        mask = df.filter(regex = i)
        mean = list(mask.filter(regex = i).columns)[0]  # because mean is vor ref values
        low = list(mask.filter(regex = 'ref').columns)[0] #columname of corresponding low ref column
        high = list(mask.filter(regex = 'ref').columns)[1] #columname of corresponding high ref column
        
        # Creating temporaly columns via boolean indexing
        # Only one per row of the temporaly made columns is filled with values (Boolean Array like)
        
        df[i+"_diff_low"] = np.where((df[mean] < df[low]),(df[low] -df[mean]), 0) 
        df[i+"_diff_high"] = np.where((df[mean] > df[high]),(df[high] -df3[mean]), 0)
        df[i+"diff_none"] = np.where((df[mean] <df[high]) & (df[mean]>df[low]),(df[mean]), 0)
        
        # Create difference column and drop all temporaliy made columns
        
        df[i+"_diff"] = df[i+"_diff_low"]+df[i+"_diff_high"]+df[i+"diff_none"]  
        df.drop([i+"_diff_low" ,i+"_diff_high",i+"diff_none"], axis = 1, inplace = True)
        

In [344]:
doubleRef(df3,RefDouble)

In [345]:
df3.filter(regex = 'chlorkonzentration')

,pa_chlorkonzentration_mean,pa_chlorkonzentration_ref_low_ppm,pa_chlorkonzentration_ref_high_ppm,pa_chlorkonzentration_diff
0,1558.000000,1560.0,1700.0,2.000000
1,1558.000000,1560.0,1700.0,2.000000
2,1558.000000,1560.0,1700.0,2.000000
3,1558.000000,1560.0,1700.0,2.000000
4,1558.000000,1560.0,1700.0,2.000000
...,...,...,...,...
118682,913.555556,1560.0,1700.0,646.444444
118683,913.555556,1560.0,1700.0,646.444444
118684,913.555556,1560.0,1700.0,646.444444
118685,913.555556,1560.0,1700.0,646.444444


# Calulate for ct1_values the diff between ref and high

In [346]:
ct1FeatureList = list(df3.filter(regex = 'ct1_').columns)[:-2]
ct1RefList = list(df3.filter(regex = 'ct1_').columns)[8:]

ct1FeatureList

['ct1_anfang_0m_%',
 'ct1_anfang_flux_m3/m2/d',
 'ct1_mitte_1_1400m_rej_%',
 'ct1_mitte_1_1400m_flux',
 'ct1_mitte_2_2900m_rej_%',
 'ct1_mitte_2_2900m_flux',
 'ct1_ende_4300m_rej_%',
 'ct1_ende_flux_m3/m2/d']

In [347]:
ct1RefList

['ct1_test_ref_low', 'ct1_test_ref_high']

In [349]:
df3.filter(regex = 'ct1')

,ct1_anfang_0m_%,ct1_anfang_flux_m3/m2/d,ct1_mitte_1_1400m_rej_%,ct1_mitte_1_1400m_flux,ct1_mitte_2_2900m_rej_%,ct1_mitte_2_2900m_flux,ct1_ende_4300m_rej_%,ct1_ende_flux_m3/m2/d,ct1_test_ref_low,ct1_test_ref_high,ct1_anfang_0m_%_diff,ct1_anfang_flux_m3/m2/d_diff,ct1_mitte_1_1400m_rej_%_diff,ct1_mitte_1_1400m_flux_diff,ct1_mitte_2_2900m_rej_%_diff,ct1_mitte_2_2900m_flux_diff,ct1_ende_4300m_rej_%_diff,ct1_ende_flux_m3/m2/d_diff
0,99.63,1.49,99.54,1.65,99.58,1.52,99.72,1.49,1.12,99.1,-0.53,1.49,-0.44,1.65,-0.48,1.52,-0.62,1.49
1,99.63,1.49,99.54,1.65,99.58,1.52,99.72,1.49,1.12,99.1,-0.53,1.49,-0.44,1.65,-0.48,1.52,-0.62,1.49
2,99.63,1.49,99.54,1.65,99.58,1.52,99.72,1.49,1.12,99.1,-0.53,1.49,-0.44,1.65,-0.48,1.52,-0.62,1.49
3,99.63,1.49,99.54,1.65,99.58,1.52,99.72,1.49,1.12,99.1,-0.53,1.49,-0.44,1.65,-0.48,1.52,-0.62,1.49
4,99.63,1.49,99.54,1.65,99.58,1.52,99.72,1.49,1.12,99.1,-0.53,1.49,-0.44,1.65,-0.48,1.52,-0.62,1.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118682,99.51,1.37,99.49,1.43,0.00,0.00,0.00,0.00,1.12,99.1,-0.41,1.37,-0.39,1.43,1.12,1.12,1.12,1.12
118683,99.51,1.37,99.49,1.43,0.00,0.00,0.00,0.00,1.12,99.1,-0.41,1.37,-0.39,1.43,1.12,1.12,1.12,1.12
118684,99.51,1.37,99.49,1.43,0.00,0.00,0.00,0.00,1.12,99.1,-0.41,1.37,-0.39,1.43,1.12,1.12,1.12,1.12
118685,99.51,1.37,99.49,1.43,0.00,0.00,0.00,0.00,1.12,99.1,-0.41,1.37,-0.39,1.43,1.12,1.12,1.12,1.12


In [348]:
for i in ct1FeatureList:
    
        low = ct1RefList[0] #columname of corresponding low ref column
        high = ct1RefList[1] #columname of corresponding high ref column
        
        # Creating temporaly columns via boolean indexing
        # Only one per row of the temporaly made columns is filled with values (Boolean Array like)
        
        df3[i+"_diff_low"] = np.where((df3[i] < df3[low]),(df3[low] -df3[i]), 0) 
        df3[i+"_diff_high"] = np.where((df3[i] > df3[high]),(df3[high] -df3[i]), 0)
        df3[i+"diff_none"] = np.where((df3[i] <df3[high]) & (df3[i]>df3[low]),(df3[i]), 0)
        
        # Create difference column and drop all temporaliy made columns
        
        df3[i+"_diff"] = df3[i+"_diff_low"]+df3[i+"_diff_high"]+df3[i+"diff_none"]  
        df3.drop([i+"_diff_low" ,i+"_diff_high",i+"diff_none"], axis = 1, inplace = True)

In [354]:
df4 = df3.copy()

In [291]:
dada = []
for i in RefSingle:
    mask = df.filter(regex = i)
    names = mask.columns
    dada.append(names)

    
flat_list = []
for sublist in dada:
    for item in sublist:
        flat_list.append(item)
print(flat_list)

['ct1_anfang_0m_%', 'ct1_anfang_flux_m3/m2/d', 'ct1_mitte_1_1400m_rej_%', 'ct1_mitte_1_1400m_flux', 'ct1_mitte_2_2900m_rej_%', 'ct1_mitte_2_2900m_flux', 'ct1_ende_4300m_rej_%', 'ct1_ende_flux_m3/m2/d', 'ct1_test_ref_low', 'ct1_test_ref_high', 'pa_airknife_vor_amin_bad_ref_%', 'pa_amin-trockner_temperatur_danfugt_c', 'pa_amin-trockner_temperatur_danfugt_ref_c', 'pa_decan-trockner_geblase_start_%', 'pa_decan-trockner_geblase_ende_%', 'pa_decan-trockner_geblase_ref_%']


# Function to calculate for single Reference value

In [355]:
restSingleVar =['pa_airknife_vor_amin_bad_mean',
                'pa_amin-trockner_temperatur_danfugt_c',  
                'pa_decan-trockner_geblase_start_%', 
                'pa_decan-trockner_geblase_ende_%', 
                ]

restSingleRef = ['pa_airknife_vor_amin_bad_ref_%',
                'pa_amin-trockner_temperatur_danfugt_ref_c',
                'pa_decan-trockner_geblase_ref_%',
                'pa_decan-trockner_geblase_ref_%']



In [368]:
c=(zip(restSingleVar,restSingleRef))
for i,j in zip(restSingleVar,restSingleRef):
    print("{},{}".format(i,j))

pa_airknife_vor_amin_bad_mean,pa_airknife_vor_amin_bad_ref_%
pa_amin-trockner_temperatur_danfugt_c,pa_amin-trockner_temperatur_danfugt_ref_c
pa_decan-trockner_geblase_start_%,pa_decan-trockner_geblase_ref_%
pa_decan-trockner_geblase_ende_%,pa_decan-trockner_geblase_ref_%


In [369]:
for a,b in zip(restSingleVar,restSingleRef):
     
    df4[a+"_diff"] = np.where((df4[a] > df4[b]) | (df4[a]<df4[b]),(df4[a]-df4[b]), 0)
        

# Turn all _diff named columns into abs() values

In [380]:
diff = list(df4.filter(regex = 'diff').columns)
for i in diff:
    df4[i] = df4[i].abs()

In [381]:
df4

,ps,ps_datum,ps_beschichtete_rollenlange_m,ps_c_losung_wt_%,ps_gap_micro_m,ps_raum_temperatur_start_c,ps_auftragsbank_temperatur_start_c,ps_raum-feuchtigkeit_start_%,ps_datum_coating,ps_out_m,ps_scrap_%,ps_f_auftragswerk_median,pa_datum,pa_beschichtete_rollenlange_m,pa_bahn-geschwindigkeit_m/min,pa_bad-wechsel_m-pda,pa_bad-wechsel_hw1,pa_bad-wechsel_hw2,pa_bad-wechsel_chlor,pa_bad-wechsel_hw3,pa_raum_temperatur_start_c,pa_temperatur_m-pda-bad_c,pa_ce-capro_lactam_%,pa_temperatur_n-decan-lsg_chem_vorbereitung_start_c,pa_ctmc_%,pa_temperatur_alkali-lsg_chem_vorbereitung_start_c,pa_temperatur_chlorbad_start_c,pa_raum_feuchte_start_%,pa_amin-trockner_temperatur_danfugt_c,pa_amin-trockner_temperatur_cofely_c,pa_decan-trockner_geblase_start_%,pa_decan-trockner_geblase_ende_%,pa_vertikale_feuchte_oben_start_%,pa_vertikale_feuchte_oben_mitte_%,pa_vertikale_feuchte_oben_ende_%,pa_staub-sauger_1_vor_aminbad_0_aus_>0_an,pa_staub-sauger_2_nach_aminbad,pa_staub-sauger_3_zw5_vor_hw2,pa_staub-sauger_4_nach_hw2,pa_staub-sauger_5,pa_coating,pa_datum_coating,pa_mpda_lsg,pa_ndecan/tmc,pa_chlorlsg,pa_sbs_lsg,pa_alkalilsg,pa_bad-wechsel_mpda_nach,pa_bad-wechsel_chlor_nach,pa_out_m,pa_scrap_allgemein_m,pa_defects_semket_#,pa_defects_dr_schenk_#,pa_defects_hand_#,pa_defects_total_#,pa_scrap_%,ct1_anfang_0m_%,ct1_anfang_flux_m3/m2/d,ct1_mitte_1_1400m_rej_%,ct1_mitte_1_1400m_flux,ct1_mitte_2_2900m_rej_%,ct1_mitte_2_2900m_flux,ct1_ende_4300m_rej_%,ct1_ende_flux_m3/m2/d,ct_retest_anfang_0m_%,ct_retest_anfang_flux_m3/m2/d,ct_retest_mitte_1_1400m_rej_%,ct_retest_mitte_1_1400m_flux,ct_retest_mitte_2_2900m_rej_%,ct_retest_mitte_2_2900m_rej_%_2,ct_retest_ende_4300m_rej_%,ct_retest_ende_flux_m3/m2/d,qt_datum,qt_roll_after_m,qt_roll_minus_scrap_meters_m,qt_scrap_%,qt_coating_scrap_%,pp_product_short_name,pp_plan_actual_datum_coating,pp_plan_product,pp_plan_ausbeute_elemente,pp_plan_end_datum_winding,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,pp_scrap_first_outer_dia_m,pp_scrap_last_core_m,pp_sequence_aw1_winding,pp_sequence_aw2_winding,pa_tmc_gehalt_%,pa_rollen_seit_letztem_bad-wechsel_mpda,pa_rollen_seit_letztem_bad-wechsel_chlor,pa_ref,pda_median,chlor_con_median,f_danfugt_bs_median,f_danfugt_as_median,f_codfely_median,temp_median,reaction_start,reaction_end,winding_pa_m,winding_scrap_first_m,winding_scrap_last_m,winding_product_short_name,winding_product_type,winding_product_size,winding_number_of_leaves,winding_product_line,winding_begin_datum,winding_end_datum,assembled_in_elements_#,assembled_in_elements_m,assembling_scrap_%,leak_test_ok_#,leak_test_ok_m,leak_test_scrap_%,winding_scrap_%,sc_d_datum,sc_datum_generate,sc_l_leak_auto,sc_l_datum_auto,sc_l_leak_hand,sc_l_datum_hand,sc_l_ergebnis_p_nio,sc_l_pressure_start,sc_l_pressure_prozent,sc_l_parameter_pressure_absolut,sc_l_parameter_pressure_prozent,sc_d_parameter_min,sc_d_parameter_max,sc_d_parameter_anz_werte,sc_d_parameter_1_min,sc_d_parameter_1_max,sc_d_parameter_deltamax,sc_d_ergebnis_min_ok,sc_d_ergebnis_max_ok,sc_d_ergebnis_anz_min_ok,sc_d_ergebnis_anz_max_ok,sc_d_ergebnis_deltamax_ok,sc_d_ergebnis_nio,sc_d_links_min,sc_d_links_max,sc_d_links_anz_min,sc_d_links_anz_max,sc_d_links_deltamax,sc_d_rechts_min,sc_d_rechts_max,sc_d_rechts_anz_min,sc_d_rechts_anz_max,sc_d_rechts_deltamax,nr,element_test_ok_#,element_test_ok_m,element_test_scrap_%,total_scrap_%,total_yield_%,qc_serien_nummer,qc_lasttest,qc_salzrckhalt,qc_durchfluss,qc_p_position,qc_erfassungs_datum,qc_verpackungs_datum,qc_einlager_datum,qc_barcode_leak_test_values,qc_datum_leak_test_values,qc_minimalersalzrueckhalt,qc_minimalerdurchfluss,qc_durchminimalersalzrueckhalt,qc_durchminimalerdurchfluss,qc_minimalerdurchflussgpd,qc_maximalersalzrueckhalt,qc_maximalerdurchfluss,qc_konzentration,qc_faktorkonzentration,qc_druckpsi,qc_temperaturc,qc_rueckhalt,qc_ph,qc_nachkommadurchfluss,qc_nachkommasalzrueckhalt,qc_rundunggpd,qc_datum_product_properties,qc_konzentrationflow,qc_lagen,qc_mstaerke,qc_breite,p_product,p_product_full_